In [1]:
import pandas as pd

In [4]:
data = pd.read_csv("/home/etienne/thesis/datasets/mammography/dataset.csv")

In [5]:
data

,age_c,assess_c,cancer_c,compfilm_c,density_c,famhx_c,hrt_c,prvmam_c,biophx_c,mammtype,CaTypeO,bmi_c,ptid
0,62,1,0,1,2,0,0,1,0,1,8,24.023544,1
1,65,1,0,1,4,0,0,1,0,1,8,-99.000000,2
2,69,0,0,1,2,0,0,1,0,1,8,29.052429,3
3,64,2,0,1,2,0,0,1,0,1,8,-99.000000,4
4,63,3,0,1,2,0,0,1,1,1,8,33.729523,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,80,1,0,1,1,0,0,1,0,1,8,-99.000000,36711
39996,78,1,0,1,3,0,0,1,0,1,8,-99.000000,36712
39997,77,1,0,1,2,0,0,1,0,1,8,-99.000000,36712
39998,66,1,0,1,2,0,0,1,1,1,8,-99.000000,36713


In [14]:
data[data["cancer_c"] == 1]

,age_c,assess_c,cancer_c,compfilm_c,density_c,famhx_c,hrt_c,prvmam_c,biophx_c,mammtype,CaTypeO,bmi_c,ptid
127,87,0,1,1,3,0,0,1,0,1,2,27.520645,116
430,76,0,1,1,2,0,0,1,1,1,2,21.632889,393
523,80,0,1,1,2,0,0,1,1,2,2,19.968811,477
947,68,0,1,1,3,0,0,1,0,2,2,-99.000000,864
963,67,4,1,1,1,0,0,1,0,2,2,-99.000000,877
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39162,78,0,1,1,3,0,0,1,1,1,2,-99.000000,35950
39430,79,0,1,1,1,1,0,1,0,1,2,27.397202,36192
39435,69,0,1,1,2,1,0,1,0,2,2,-99.000000,36196
39500,60,0,1,1,2,0,0,1,0,1,1,29.284851,36258
